## Code

In [1]:
%load_ext ipython_sparql_pandas

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
from random import randint
sparql = SPARQLWrapper('http://MacBook-Air-van-Daphne.local:7200/repositories/statements')
sparql.setReturnFormat(JSON)

In [3]:
#from the query results extract a list that contains the counts after splitting on certain attributes
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
NegAttr=[]
AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']

def extractCountAndAttr(result):
    number = []
    listOfObj = []
    listOfPred= []
    #ret = result.queryAndConvert()
    for attr in result["results"]["bindings"]:
        listOfObj.append(attr['o'])#["value"])
        listOfPred.append(attr['p'])#["value"])
    return(listOfObj,listOfPred)

def generateQuestion(listOfPred, listOfObj, index, subject):
    flag = True
    if ('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>') in AttrHistory:
        return generateQuestion(listOfPred, listOfObj, index+1, subject)
        
    else:
        AttrHistory.append('<' + str(listOfPred[index]['value']) +'> <'+str(listOfObj[index]['value'])+ '>')
        
    try:
        int(listOfObj[index]['value'][0])
    except ValueError:
           flag = False
        
    #print(f'Does the thing you are looking for have the attribute: {listOfPred[index]["value"].split("/")[-1]} {listOfObj[index]["value"].split("/")[-1]}?' )
    
    if flag is True:
        datatype = listOfObj[index]["datatype"].split("#")[-1]
        answer = findAnswer(subject, '<' + str(listOfPred[index]["value"]) +'>', '"'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype)
    else:
        answer = findAnswer(subject, '<' + str(listOfPred[index]["value"]) +'>', '<'+str(listOfObj[index]["value"])+ '>')
    #print(answer)
    
    if answer:
        #PosAttr.append('?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>.')    
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'.') 
        else:
            PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>.')
        
    elif not answer:
        #NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>. }}')   
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'. }}')
        else:
            NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>. }}')   

    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    return (PosFilters, NegFilters)

  
def updateQuery(left, PosFilters, NegFilters):
    FilterQuestion =  (f"""
            select (count(*) as ?count) ?p ?o  where 
            {{
            
            {PosFilters}
             ?s ?p ?o .
            {NegFilters}
            }} 
            
            group by ?p ?o 
            ORDER BY ABS( {left} - ?count )
            Limit 5
            
            """) 
        
    return FilterQuestion

In [4]:
def pickSample():
    query = f"""
    select distinct ?s
    where {{
        ?s ?p ?o .
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
    }}
    offset 5678
    limit 1000
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    return qres["results"]["bindings"]

In [5]:
def findAnswer(subject,p,o):
    query = f"""
        SELECT ?s ?p ?o
        WHERE {{
        ?s ?p ?o .
            FILTER(?s = {subject})
            FILTER(?p = {p})
            FILTER(?o = {o})
            }}
            """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    res = qres["results"]["bindings"]
    return (len(res))>0

In [6]:
def numberleft(PosFilters,NegFilters):
    query =  f"""
            select (count(?s) as ?count) where 
            {{
            {PosFilters}            
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
          }}       
            group by ?p ?o 
            ORDER BY DESC(?count )            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        left= (r["count"]["value"])
        return(int(left))

In [7]:
def popentities(PosFilters,NegFilters):
    query =  f"""
            select ?s where 
            {{
            
            {PosFilters}
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
    
            }}
            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    return qres["results"]["bindings"]

In [8]:
# resets all variables
def game_reset():
    PosAttr= ['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
    NegAttr= ['']
    AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']
    left = numberleft("".join(PosAttr), "".join(NegAttr))
    FilterQuestion =  f"""
        select (count(*) as ?count) ?p ?o  where 
        {{

        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
        ?s ?p ?o .
        }} 
        group by ?p ?o 
        ORDER BY ABS( %s - ?count )
        limit 5
        """ %((left/2))
    sparql.setQuery(FilterQuestion)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    listOfObj ,listOfPred = extractCountAndAttr(qres)
    return (left, qres, PosAttr, NegAttr, AttrHistory)

In [10]:
#without printing results
import random

wincount = 109
gamesplayed = 176
totalgames = 1000
sum_avg_questions = 1576

sample = []
sample_dict = pickSample()
for subject in sample_dict:
    sample.append(subject["s"]["value"])

while gamesplayed < totalgames:
    # Resets all variables
    left,qres,PosAttr,NegAttr,AttrHistory = game_reset()
    i=0
    outofguess = []
    #print("\nGames played: " + str(gamesplayed))
    
    # pick subject
    subject = ("<" + str(sample[gamesplayed]) + ">")
    #print("\n" + subject)

    while i < 20:

        # list of Pred and Obj for questions
        listOfObj ,listOfPred = extractCountAndAttr(qres)


        # if only 1 popular entity is left break
        if left<=1:
            break

        # ask question and add filters to query
        #print('Question: %s' %(i+1))
        try:
            PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, 0, subject)
            exception_occured = False
        except: 
            exception_occured = True
            gamesplayed += 1
            print("Query took too long\nCorrect: " + str(wincount) + "/" + str(gamesplayed) + " (average: " + str(avg_questions) + " questions)")
            break

        # number of popular entities left
        left = numberleft(PosFilters, NegFilters)

        # run query
        query = updateQuery(left/2,PosFilters, NegFilters)
        sparql.setQuery(query)
        qres = sparql.queryAndConvert()

        i+=1
        
    if exception_occured:
        continue

    # Prints answer
    if i != 20:
        for attr in popentities(PosFilters, NegFilters):
            for x in [attr['s']['value']][0:10]:
                answer = (x.split("/")[-1])
        #print("Your Answer is: " + answer)
        if str(answer.split("/")[-1] + ">") == str(subject.split("/")[-1]):
            wincount += 1
            gamesplayed += 1
            sum_avg_questions += i
            avg_questions = sum_avg_questions/wincount
            #print("Win!")
        else:
            gamesplayed += 1
            #print("Lose...")
        print("Correct: " + str(wincount) + "/" + str(gamesplayed) + " (average: " + str(avg_questions) + " questions)")

    # ran out of questions
    else:
        #print('Final Question:')
        for j in popentities(PosFilters, NegFilters):
                outofguess.append(f'{j["s"]["value"].split("/")[-1]}')
        guess = random.choice(outofguess)
        #print("Is it " + guess + "?")
        if str(subject.split("/")[-1]) == str(guess + ">"):
            wincount += 1
            gamesplayed += 1
            sum_avg_questions += i
            avg_questions = sum_avg_questions/wincount
            #print('Win!')
        else:
            gamesplayed += 1
            #print('Lose...')
        print("Correct: " + str(wincount) + "/" + str(gamesplayed) + " (average: " + str(avg_questions) + " questions)")
            
print("\nTournament ended! I have guessed " + str(wincount) + " out of " + str(gamesplayed) + " things correctly (average: " + str(avg_questions) + " questions).")

Correct: 110/177 (average: 14.454545454545455 questions)
Correct: 111/178 (average: 14.45045045045045 questions)
Correct: 111/179 (average: 14.45045045045045 questions)
Correct: 112/180 (average: 14.428571428571429 questions)
Correct: 113/181 (average: 14.47787610619469 questions)
Correct: 114/182 (average: 14.5 questions)
Correct: 115/183 (average: 14.530434782608696 questions)
Correct: 116/184 (average: 14.508620689655173 questions)
Correct: 117/185 (average: 14.478632478632479 questions)
Correct: 118/186 (average: 14.474576271186441 questions)
Correct: 119/187 (average: 14.521008403361344 questions)
Correct: 120/188 (average: 14.5 questions)
Correct: 121/189 (average: 14.512396694214877 questions)
Correct: 122/190 (average: 14.51639344262295 questions)
Correct: 123/191 (average: 14.528455284552846 questions)
Correct: 124/192 (average: 14.540322580645162 questions)
Correct: 125/193 (average: 14.56 questions)
Correct: 125/194 (average: 14.56 questions)
Correct: 126/195 (average: 14.53